# Compare builidng footprints

Compare the newly released Google footprints dataset to the DigitizeAfrica data

https://sites.research.google/open-buildings/


In [64]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd
import GOSTRocks.rasterMisc as rMisc

from GOSTRocks.misc import tPrint
from shapely.geometry import Point
from shapely.wkt import loads

In [5]:
google_file = '/home/wb411133/temp/0fd_buildings.csv.gz'
inG = pd.read_csv(google_file)
inG['geometry'] = inG['geometry'].apply(lambda x: loads(x))
inGeom = gpd.GeoDataFrame(inG, geometry='geometry', crs='epsg:4326')

In [19]:
da_file =  '/home/public/Data/COUNTRY/GHA/Buildings/120519/AFRICA_GHANA_building_32630.shp'
inD = gpd.read_file(da_file)

In [22]:
# limit the datasets to the select AOI
aoi_file = 'Data/ACCRA_AOI.shp'
inA = gpd.read_file(aoi_file)

In [24]:
inG_aoi = inA.to_crs(inGeom.crs)

In [25]:
selG = inGeom.loc[inGeom.intersects(inG_aoi.unary_union)]

In [26]:
inD_aoi = inA.to_crs(inD.crs)
selD = inD.loc[inD.intersects(inD_aoi.unary_union)]

In [29]:
out_folder = '/home/wb411133/temp/B_comparison'
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
    
selG.to_file(os.path.join(out_folder, "google.shp"))
selD.to_file(os.path.join(out_folder, "da.shp"))

<ipython-input-29-cefd2e42a7bc>:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  selG.to_file(os.path.join(out_folder, "google.shp"))


# Rasterize buildings

In [30]:
dir(rMisc)

['Affine',
 'BoundaryNorm',
 'Counter',
 'ListedColormap',
 'MemoryFile',
 'MergeAlg',
 'Resampling',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'affine',
 'box',
 'clipRaster',
 'contextmanager',
 'create_rasterio_inmemory',
 'curPath',
 'features',
 'gpd',
 'groupJaccard',
 'inspect',
 'jaccardIndex',
 'json',
 'logging',
 'map_viirs',
 'mask',
 'np',
 'os',
 'pd',
 'plt',
 'polygonizeArray',
 'pyproj',
 'rasterio',
 'rasterize',
 'rasterizeDataFrame',
 'reproject',
 'runAllJaccard',
 'rxr',
 'sns',
 'standardizeInputRasters',
 'sys',
 'tPrint',
 'wkt',
 'xr',
 'zonalResult',
 'zonalStats']

In [31]:
inD_pt = selD.copy()
inD_pt['geometry'] = inD_pt['geometry'].apply(lambda x: x.centroid)
inD_pt.head()

,PolyDate,PolySource,ImgSource,ImgDate,MosaicName,geometry
3,2019-11,Ecopia,Maxar,2017-12,19Q4_CUSTOM_GHANA_BBA,POINT (793938.654 676179.729)
14,2019-11,Ecopia,Maxar,2018-10,19Q4_CUSTOM_GHANA_BBA,POINT (809652.182 627334.699)
19,2019-11,Ecopia,Maxar,2017-02,19Q4_CUSTOM_GHANA_BBA,POINT (821129.395 671397.811)
25,2019-11,Ecopia,Maxar,2009-11,19Q4_CUSTOM_GHANA_BBA,POINT (805731.401 660383.571)
26,2019-11,Ecopia,Maxar,2018-10,19Q4_CUSTOM_GHANA_BBA,POINT (810550.634 630175.291)


In [32]:
inG_pt = selG.copy()
inG_pt['geometry'] = inG_pt['geometry'].apply(lambda x: x.centroid)
inG_pt.head()

,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
3,5.737080,-0.304698,15.3569,0.8040,POINT (-0.30470 5.73708),6CQXPMPW+R4M8
11,5.586767,-0.293191,7.6802,0.6689,POINT (-0.29319 5.58677),6CQXHPP4+PP3Q
13,5.655424,-0.004686,9.0518,0.6443,POINT (-0.00469 5.65542),6CQXMX4W+548R
20,5.693518,-0.268852,8.4102,0.7547,POINT (-0.26885 5.69352),6CQXMPVJ+CF2P
22,5.267646,-0.887453,17.3254,0.6056,POINT (-0.88745 5.26765),6CQX7497+323W


In [38]:
inG_pt = inG_pt.to_crs(inD_pt.crs)

In [75]:
res = 100

out_google = os.path.join(out_folder, f"google_buildings_{res}.tif")
gR = rMisc.rasterizeDataFrame(inG_pt, out_google, mergeAlg='ADD', res = res)

out_da = os.path.join(out_folder, f"da_buildings_{res}.tif")
daR = rMisc.rasterizeDataFrame(inD_pt, out_da, mergeAlg='ADD', templateRaster = out_google)

# Calculate differences between rasterized solutions
diff_image = gR['vals'] - daR['vals']
with rasterio.open(os.path.join(out_folder, f"bldg_difference_{res}.tif"), 'w', **gR['meta']) as outR:
    outR.write_band(1, diff_image)

{'init': 'epsg:32630'}
